<a href="https://colab.research.google.com/github/dariashcherbakovaaa/Algorithms-for-massive-data/blob/main/Algorithms_for_massive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment settings & libraries

In [1]:
import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('PageRank').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
import pyspark
type(spark)

sc = spark.sparkContext
spark

from google.colab import userdata
os.environ['KAGGLE_USERNAME'] = userdata.get("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = userdata.get("KAGGLE_KEY")

In [ ]:
# LinkedIn dataset (Finding similar items, Market-basket analysis)
# !kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024
# !unzip 1-3m-linkedin-jobs-and-skills-2024.zip -d job_skills

# *The task is to implement a ranking system based on the PageRank index using the «Prado Museum Pictures» dataset, published on Kaggle under the MIT license. The entities to be ranked are pictures, linked together if they share at least a common tag in the work_tag attribute of the  prado.csv file, although you can experiment with different strategies in order to define links between pictures.*

Dear professor, this code on dowloading and unzipping data is pretty slow (~10-20 mins), im sorry. 🥺👉👈

In [2]:
# Link analysis (PageRank)
!kaggle datasets download -d maparla/prado-museum-pictures -f prado.csv -p /content/
!unzip prado-museum-pictures.zip -d prada

 27% 5.00M/18.3M [00:00<00:00, 50.2MB/s]
100% 18.3M/18.3M [00:00<00:00, 101MB/s] 
Archive:  prado-museum-pictures.zip
replace prada/images/images/._000de341-acd0-4338-b6db-ce4127fa5b37.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: No
No
No


-- just to check how it should be seen

In [3]:
import pandas as pd

df = pd.read_csv('/content/prada/prado.csv')
df.head(3)

,work_url,work_image_url,author,author_bio,author_url,author_id,work_title,work_subtitle,work_exposed,work_description,...,bibliography,inventory,expositions,ubication,technical_sheet_autores,technical_sheet_edicion_/_estado,technical_sheet_materia,technical_sheet_ceca,technical_sheet_autora,technical_sheet_lugar_de_produccion
0,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Bayeu y Subías, Francisco","Zaragoza, 1734 - Madrid, 1795\n\nSu formación ...",https://www.museodelprado.es/coleccion/artista...,b218fee4-053b-4656-8577-9aa001ad1989,Cabeza de Gigante,"1764. Lápiz negro sobre papel verdoso, 572 x 3...",No expuesto,"De tres cuartos, mirando hacia la derecha, en ...",...,"Arnáez, Rocío, Dibujos de Francisco Bayeu en ...","Inv. Real Museo, Sección Dibujos, 1857.\nNúm. ...",[… + escudo] (Sello: Dirección del Museo Nacio...,Dibujos de Francisco Bayeu\n ...,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Crespi, Giuseppe Maria","Bolonia, 1665 - Bolonia, 1747\n\nDiscípulo de ...",https://www.museodelprado.es/coleccion/artista...,b20a5ee9-2ebd-4e60-aadc-e91e7867cb12,Susana y los viejos,"Siglo XVIII. Sanguina sobre papel agarbanzado,...",No expuesto,"A la derecha, Susana, sentada en una fuente de...",...,"Mena Marqués, Manuela, Catálogo de Dibujos. V...","Inv. Legado Pedro Fernández Durán, 1931.\nNúm....",Veneziano\nReverso del segundo soporte,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Cronenburch, Adriaen van","Schagen (Países Bajos), 1520 - Bergum (Países ...",https://www.museodelprado.es/coleccion/artista...,26861819-ff88-4fde-8a37-56db9e1c1ba4,Dama con una flor amarilla,"Hacia 1567. Óleo sobre tabla, 107 x 79 cm",No expuesto,"Esta obra, junto a sus compañeras (P02074, P02...",...,"Madrazo, Pedro de, Catálogo de los cuadros de...","Catálogo Museo del Prado, 1873-1907.\nNúm. 130...","Aaaa[""a mas tres(dri)aes"", jeroglífico de ""Adr...",El retrato del Renacimiento\n ...,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.work_url.head(10)

0    https://www.museodelprado.es/coleccion/obra-de...
1    https://www.museodelprado.es/coleccion/obra-de...
2    https://www.museodelprado.es/coleccion/obra-de...
3    https://www.museodelprado.es/coleccion/obra-de...
4    https://www.museodelprado.es/coleccion/obra-de...
5    https://www.museodelprado.es/coleccion/obra-de...
6    https://www.museodelprado.es/coleccion/obra-de...
7    https://www.museodelprado.es/coleccion/obra-de...
8    https://www.museodelprado.es/coleccion/obra-de...
9    https://www.museodelprado.es/coleccion/obra-de...
Name: work_url, dtype: object

In [4]:
prado_museum = spark.read.format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('path', '/content/prada/prado.csv') \
    .load()

prado_museum.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+------------+--------------------+-----------------+----------------------------------+---------------------+----------------------+---------------------+-----------------------+-----------------------+-------------------------+---------------------+---------------------------+------------+---------+-----------+---------+-----------------------+--------------------------------+-----------------------+--------------------+----------------------+-----------------------------------+
|            work_url|      work_image_url|              author|          author_bio|          author_url|  author_id|          work_title|       work_subtitle|work_exposed|    work_description|        work_tags|technical_sheet_numero_de_catalogo|technical_sheet_autor|technical_sheet_titulo|technical_sheet_fecha|technical_sheet_tecnica|technical_sheet_s

In [6]:
prado_museum.select('work_url').show(10)

+--------------------+
|            work_url|
+--------------------+
|https://www.museo...|
|Su formación se i...|
|Fuente: Museo Nac...|
|      Ancho: 348 mm"|
|                 @@@|
|              Arnáez|
|                 @@@|
|                Bray|
|           Núm. 135.|
|135. Ochenta y do...|
+--------------------+
only showing top 10 rows



In [5]:
prado_museum.printSchema()

root
 |-- work_url: string (nullable = true)
 |-- work_image_url: string (nullable = true)
 |-- author: string (nullable = true)
 |-- author_bio: string (nullable = true)
 |-- author_url: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- work_title: string (nullable = true)
 |-- work_subtitle: string (nullable = true)
 |-- work_exposed: string (nullable = true)
 |-- work_description: string (nullable = true)
 |-- work_tags: string (nullable = true)
 |-- technical_sheet_numero_de_catalogo: string (nullable = true)
 |-- technical_sheet_autor: string (nullable = true)
 |-- technical_sheet_titulo: string (nullable = true)
 |-- technical_sheet_fecha: string (nullable = true)
 |-- technical_sheet_tecnica: string (nullable = true)
 |-- technical_sheet_soporte: string (nullable = true)
 |-- technical_sheet_dimension: string (nullable = true)
 |-- technical_sheet_serie: string (nullable = true)
 |-- technical_sheet_procedencia: string (nullable = true)
 |-- bibliography: st

In [ ]:
prado = prada_museum.select(['work_url', 'work_tags'])
prado

work_url,work_tags
https://www.museo...,null
Su formación se i...,null
Fuente: Museo Nac...,Cabeza de Gigante
"Ancho: 348 mm""",null
@@@,null
Arnáez,null
@@@,null
Bray,null
Núm. 135.,null
135. Ochenta y do...,null


In [ ]:
pr = prada_museum.select('work_image_url').rdd.map(lambda row: row['work_image_url'].split('/')[-1]).collect()

In [ ]:
for _, row in df.iterrows():
    image_id = row["work_id"]
    image_path = os.path.join("images", image_id)

## Graph creation and visualisation

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

g = nx.DiGraph()

for p in pages:
    g.add_node(p)

for (a, b) in links:
    g.add_edge(pages[a], pages[b])

Build the transition matrix (the base to perform PageRank)